In [1]:
#to open cred files
import os

#Data transformation
import pandas as pd

#solve problems with reords
import pandas_gbq

#bigquery
from google.cloud import bigquery

#FIREBASE
import firebase_admin

from firebase_admin import credentials
from firebase_admin import firestore

from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/miong/.env')

print('beginning etl of accounting ... ')

#OPEN CONECTION TO FIREBASE
filename=os.environ['FIREBASE_MIONG_FILENAME']

beginning etl of accounting ... 


In [2]:
## OPEN CONECTION TO BIGQUERRY
bgq_client = bigquery.Client()
dataset_id = "{}.firestore_export".format(bgq_client.project)
dataset = bigquery.Dataset(dataset_id)

In [3]:
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [4]:
#get all the collection of OAN
col_query = db.collection('accountingItems').stream()

In [5]:
#save it in a list of dictionaries
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [6]:
def convData(acc):
    for item in acc['context']:
        acc[item] = acc['context'][item]
    return acc

In [7]:
accounting = []
for acc in accounts:
    accounting.append(convData(acc))

In [8]:
#convert to dataframe
accounting = pd.DataFrame(accounts)

In [9]:
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4005 entries, 0 to 4004
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   code                     4005 non-null   int64  
 1   baseAmount               4005 non-null   float64
 2   originIntervention       3985 non-null   object 
 3   type                     4005 non-null   object 
 4   responsible              4005 non-null   object 
 5   originAccountingAccount  4005 non-null   object 
 6   targetAccountingAccount  4005 non-null   object 
 7   description              3916 non-null   object 
 8   targetIntervention       3991 non-null   object 
 9   amount                   4005 non-null   float64
 10  files                    654 non-null    object 
 11  targetPhase              1703 non-null   object 
 12  vatAmount                4005 non-null   float64
 13  targetProject            3991 non-null   object 
 14  concept                 

In [36]:
df = accounting[['id','account','createdByType','createdBy','lastUpdateBy',
 'createdAt','lastUpdateAt','code','executedAt','responsible',
 'concept','description','type','originProject','originIntervention','originPhase',
 'targetProject','targetIntervention','tags','originAccountingAccount',
 'targetAccountingAccount','amount','baseAmount','vat','vatAmount','files']].copy()

In [47]:
for a in df.columns:
    if a in ['id','account','createdByType','createdBy','lastUpdateBy',
 'createdAt','lastUpdateAt','executedAt','responsible',
 'concept','description','type','originProject','originIntervention','originPhase',
 'targetProject','targetIntervention','originAccountingAccount',
 'targetAccountingAccount']:
    
        df[a] = df[a].astype(str)
    elif a in ['files','tags']:
        df[a] = df[a].apply(lambda x: x if x else [])
    elif a == 'code':
        df[a] = df[a].apply(int)
    else:
        df[a] = df[a].apply(lambda x: float(x) if x 
                                               else 0.0 if x == ''
                                               else 0.0)
        
        

In [48]:
table_id = "{}.accountingItems".format(dataset.dataset_id)
proj = bgq_client.project

In [50]:
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("tags", "STRING",mode='REPEATED',),
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("files", "STRING",mode='REPEATED'),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = bgq_client.load_table_from_dataframe(
    df, table_id, project=bgq_client.project,
    job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

In [23]:


pandas_gbq.to_gbq(
    accounting, table_id, project_id=bgq_client.project, if_exists='replace',
)

print('done.')

1it [00:11, 11.75s/it]

done.


In [16]:
accounting.columns

Index(['targetAccountingAccount', 'originProject', 'responsible', 'code',
       'context', 'type', 'targetProject', 'originAccountingAccount',
       'description', 'originIntervention', 'tags', 'targetPhase', 'concept',
       'files', 'baseAmount', 'amount', 'executedAt', 'vat', 'vatAmount',
       'targetIntervention', 'originPhase'],
      dtype='object')

In [ ]:
id	STRING	NULLABLE	
account	STRING	NULLABLE	
createdByType	STRING	NULLABLE	
createdBy	STRING	NULLABLE	
lastUpdateBy	STRING	NULLABLE	
createdAt	STRING	NULLABLE	
lastUpdateAt	STRING	NULLABLE	
code	INTEGER	NULLABLE	
executedAt	STRING	NULLABLE	
responsible	STRING	NULLABLE	
concept	STRING	NULLABLE	
description	STRING	NULLABLE	
type	STRING	NULLABLE	
originProject	STRING	NULLABLE	
originIntervention	STRING	NULLABLE	
originPhase	STRING	NULLABLE	
targetProject	STRING	NULLABLE	
targetIntervention	STRING	NULLABLE	
tags	STRING	REPEATED	
originAccountingAccount	STRING	NULLABLE	
targetAccountingAccount	STRING	NULLABLE	
amount	FLOAT	NULLABLE	
baseAmount	FLOAT	NULLABLE	
vat	FLOAT	NULLABLE	
vatAmount	FLOAT	NULLABLE	
files	RECORD	REPEATED	
